### Scraping tool to get reviews from restaurants. Firstly the function scrapeRestaurantsUrlsAll is used to gether all the restaurant urls based on location. Then we iterate through those urls and collect all the reviews.  

In [7]:
import requests 
from bs4 import BeautifulSoup
import csv 
from selenium import webdriver
import time
import sys
import os
import argparse
import string
import pickle

In [14]:
pathToReviews = "TripReviews.csv"
pathToStoreInfo = "TripStoresInfo.csv"
pathAllRestaurants = "AllRestaurants.txt"

# with open(pathToStoreInfo, mode='a', encoding="utf-8") as trip:
#     data_writer = csv.writer(trip, delimiter = ',', quotechar = '"', quoting = csv.QUOTE_MINIMAL)
#     data_writer.writerow(['storeName', 'storeAddress', 'avgRating', 'nrReviews', 'priceCategory','CousineType', 'Rank'])
# #webDriver init

with open("TripReviews620-720.csv", mode='a', encoding="utf-8") as trip_data:
    data_writer = csv.writer(trip_data, delimiter = ',', quotechar = '"', quoting = csv.QUOTE_MINIMAL)
    data_writer.writerow(['storeName', 'reviewerUsername', 'ratingDate', 'reviewHeader','reviewText', 'rating'])

In [3]:
# Add urls for all "next" pages
def scrapeRestaurantsUrls(tripURLs):
    urls =[]
    for url in tripURLs:
        page = requests.get(url)
        soup = BeautifulSoup(page.text, 'html.parser')
        results = soup.find('div', class_='_1kXteagE')
        stores = results.find_all('div', class_='wQjYiB7z')
        for store in stores:
            unModifiedUrl = str(store.find('a', href=True)['href'])
            urls.append('https://www.tripadvisor.com'+unModifiedUrl)
    return urls

# Add urls for all "next" pages
def scrapeRestaurantsUrlsAll(url, limit=100):
    store_name = []
    urls = []
    limit_set = 1
    nextPage = True
    while nextPage and limit_set <= limit:
        page = requests.get(url)
        soup = BeautifulSoup(page.text, 'html.parser')
        results = soup.find('div', class_='_1kXteagE')
        stores = results.find_all('div', class_='wQjYiB7z') 
        for store in stores:
            if store.find('a', class_ = '_15_ydu6b').text[0].isdigit(): # skip the ones that er sponsored since they will also come later.
                
                print(store.find('a', class_ = '_15_ydu6b').text)
                unModifiedUrl = str(store.find('a', href=True)['href'])
                urls.append('https://www.tripadvisor.com'+unModifiedUrl)
        limit_set += 1
        #Go to next page if exists
        try:
            print('tried next in finding all')
            unModifiedUrl = str(soup.find('a', class_ = 'nav next rndBtn ui_button primary taLnk',href=True)['href'])
            # print(unModifiedUrl, 'later unmod')
            url = 'https://www.tripadvisor.com' + unModifiedUrl
            # print('new url is ', url)
        except:
            print('no next in finding all')
            nextPage = False

    with open(pathAllRestaurants, 'wb') as f:
        pickle.dump(urls, f)

    print(f'Total restaurant count: {len(urls)}')
    return urls

def scrapeRestaurantInfo(url):
    page = requests.get(url)
    soup = BeautifulSoup(page.text, 'html.parser')
    storeName = soup.find('h1', class_='_3a1XQ88S').text
    avgRating = soup.find('span', class_='r2Cf69qf').text.strip()
    storeAddress = soup.find('div', class_= '_2vbD36Hr _36TL14Jn').find('span', class_='_2saB_OSe').text.strip()
#     urlAddress = str(soup.find('div', class_ = '_2vbD36Hr _36TL14Jn').find('span').find('a', href=True)['href'])
    nrReviews = soup.find('a', class_='_10Iv7dOs').text.strip().split()[0]
    cousineType = [word.text for  word in soup.find('span', class_='_13OzAOXO _34GKdBMV').find_all('a')]
    nrPos = soup.find('a', class_='_15QfMZ2L').find('b').find('span').text.strip()
    with open(pathToStoreInfo, mode='a', encoding="utf-8") as trip:
        data_writer = csv.writer(trip, delimiter = ',', quotechar = '"', quoting = csv.QUOTE_MINIMAL)
        data_writer.writerow([storeName, storeAddress, avgRating, nrReviews, cousineType[0], cousineType[1:], nrPos])

In [19]:
# # If used with terminal directly the parser defines the values magnitude that is wanted to scrape.
# parser = argparse.ArgumentParser()
# parser.add_argument('--url', required=True, help ='need starting url')
# parser.add_argument('-i', '--info', action='store_true', help="Collects restaurant's info")
# parser.add_argument('-m', '--many', action='store_true', help="Collects whole area info")
# parser.add_argument('-l', '--limit', help="Set a limit on how many restaurant pages when used with area")
# args = parser.parse_args()
# startingUrl = args.url 
# if args.info:
#     info = True
# else:
#     info = False
# if args.many:
#     if args.limit:
#         print(f'The limit on pages is: {args.limit}')
#         urls = scrapeRestaurantsUrlsAll(startingUrl)
#     else:
#         urls = scrapeRestaurantsUrls([startingUrl])
# else:
#     urls = [startingUrl]

In [28]:
startingUrl = "https://www.tripadvisor.com/Restaurants-g189541-Copenhagen_Zealand.html"
limit = 100
# urls = scrapeRestaurantsUrlsAll(startingUrl, limit)

1. Maple Casual Dining
2. Keyser Social
3. Restaurant Krebsegaarden
4. The Olive Kitchen & Bar
5. Ark
6. Madbaren Marmorkirken
7. Smagsloeget (Vesterbro)
8. The Shrimp
9. The Midwestern Diner
10. Zahida
11. Pizzeria Mamemi & Wine Bar
12. Soelleroed Kro
13. Punk Royale
14. Limone Piazza
15. Jurfood
16. The Pescatarian
17. Hey Darling
18. Alchemist
19. Marv & Ben
20. Enomania
21. Restaurant Komplet
22. Souls
23. Geranium
24. Banana Joe
25. Trattoria SUD
26. Mielcke & Hurtigkarl
27. Cafe Patina
28. Restaurant Schonnemann
29. Husmann's Vinstue
30. Restaurant leaven
tried next in finding all
31. The Red Box
32. Restaurant Gronnegade
33. Studio - The Standard
34. Kong Hans Kaelder
35. Basso Kobenhavn
36. Host
37. Che Fe Trattoria
38. Restaurant Frederiks Have
39. Ravnsborg Kitchen & Bar
40. Temporada Tapas-Bar
41. Chao Viet Kitchen
42. Restaurant Tight
43. Feed Bistro
44. Bowl Market Copenhagen
45. Cofoco
46. Bastard Cafe
47. Restaurant Nepali Bhancha
48. Restaurant Amalie
49. Noma
50. Garde

391. Hachi Sushi
392. Hope Organic
393. Gorm's Nyhavn
394. Thirion - Charcuterie & Vins
395. Le Petit i Torvehallerne
396. Restaurant Dubrovnik
397. Beyzade
398. Gemyse
399. Cafe Gavlen
400. Tapas Huset Casa Lola
401. Hidden Dimsum
402. Brooklyn Bar Koebenhavn
403. Cantina
404. The Dubliner
405. Ewalds
406. Astor Deep Pan Pizza
407. Boltens Food Court
408. The Globe Irish Pub
409. Grillen Burgerbar Christianshavn
410. Hanzo
411. Falernum
412. Le Caprice
413. La La La
414. Teaterkaelderen
415. Thai Asia
416. Voraz Bar & Cafe
417. Bonjour Vietnam
418. Cafe G
419. Dag H
420. Blue Bike Cafe
tried next in finding all
421. Pizza Time
422. Urten
423. Yaffa Restaurant & Bar
424. Restaurant Taller
425. The Modern Cafe & Brasserie
426. Bar'Vin
427. The Old English Pub
428. The Bird & The Churchkey
429. Gronsagen
430. Ravnsborg Vinbar
431. Boston Grill Mayfair
432. Christianshavns Badudlejning & Cafe
433. Ambassaden & Konsulatet
434. Ingolfs Kaffebar
435. pizzeria Venezia Nyhavn 31
436. Ristorant

781. Ranee's
782. L'Anfora
783. Bombay
784. Izumi Vesterbro
785. Pixie
786. Cafe Ermanno
787. MD Thai
788. Gorm's i Tivoli
789. The Grill
790. Oki Sushi
791. Kodbyens IS
792. Terroiristen
793. Pub & Sport
794. Poco Picante
795. Sushi2500
796. Lille Bakery
797. Kiin Kiin 71 Omakase
798. Pio Vin & Bar
799. Cheung's Grill
800. Le Kaff
801. Taim Restaurant
802. Mudan Restaurant
803. Cafe Alma
804. Restaurant Romarin
805. Djuus Copenhagen
806. Ancestrale
807. Cafe Fata'Morgana
808. TGI Fridays
809. La Vecchia Italia
810. Wagamama
tried next in finding all
811. Wannisa
812. The Black Cock
813. Kind of Blue
814. Spicey Kitchen
815. Restaurant Barshu
816. Falang Asiateria
817. Cafe Hoegs Hoegs
818. Coffee Fellaz
819. Hereford Village
820. Greasy Spoon
821. Chun bo
822. Kiin Kiin Street at WestMarket
823. Maltid
824. Cafe Bacchouse
825. Hanaya
826. Milano Restaurant & Pizzaria
827. Restaurant Porn-Sak
828. The a la menthe
829. Henckell
830. Graesk Smag
831. Bombay Bistro
832. Hos Fischer
833. S

1171. Massimo - Italiensk Pizzeria
1172. Meyers Deli
1173. Rays
1174. Logismose Vin, Mad & Delikatesser
1175. The Secret Mad & Vin
1176. Chaophraya
1177. Domo Restaurant
1178. Magasasa
1179. Norbro Kaffebar
1180. Cafe Stadion
1181. Ristorante Uno
1182. Vivah
1183. Pizzeria Kanalen
1184. Copenhagen Smorrebrod
1185. Cafe Bjerget
1186. Il Gusto Giusto
1187. Asmara
1188. Kaffetarnet
1189. No. 4 Ny Nordisk Tapas & Vinbar
1190. The Irish Bull
1191. Brasserie Murdoch's Books & Ale
1192. Cafe Grekenlandsvej
1193. Iki Sushi Ndr. Frihavnsgade
1194. So Last Year 2nd Hand & Coffee
1195. Thaiaway
1196. Colpo Grosso
1197. The Corner
1198. Thai Kacha
1199. Ambassadeur
1200. Jagger Fast Food
tried next in finding all
1201. Kaffebaren Pa Amager
1202. Hotfodogs
1203. Chopstix
1204. Mamma Mokka extra
1205. Viadukt Cafe & Restaurant
1206. Brasserie du Nord
1207. Royal Bagel Copenhagen
1208. Shahi Restaurant
1209. DAC Cafe
1210. The Mudhoney
1211. My Poke Illum
1212. hing's sushi
1213. Cafe Mambo
1214. Cap

1561. Como
1562. Il Corvino
1563. Frankies Pizza Frederiksberg
1564. Barlie
1565. Dalle Valle Amager
1566. D.D. Thai Take Away
1567. Botanisk Haves Cafe
1568. Letz Sushi Nordhavn
1569. Tove Smorrebrod
1570. The Organic LAB
1571. Shabaz Kaffebar & Kokken
1572. Madhuset
1573. 7 Eleven
1574. Tex Mex
1575. Cafe Maroc
1576. Blue Raven
1577. Opera Cafe
1578. Vinoria Lilo
1579. Restaurant Indgang
1580. Bar Rouge
1581. Koi Sushi
1582. Baguette Company
1583. Pizzeria Leo - Solvgade
1584. Pronto Pizza III
1585. Engell & Solovej
1586. Her Er Ro Pub
1587. Gastronetten Smorrebrod
1588. Sansab
1589. Sorte Firkant
1590. Hermes Antik & Kaffebar
tried next in finding all
1591. Polsevognen ved Norrebro Runddel
1592. Gastronomia Candido
1593. Lobster Shack
1594. Couloir
1595. Izumi Osterbro
1596. Lagkagehuset
1597. Det Rene Brod
1598. Tempo
1599. El Porron
1600. Grabs Deli
1601. Cafe Oum
1602. Spaghetti Martelli
1603. Bike and Bake
1604. JAGGER Fast Food
1605. Pandekagehuset Norrebro
1606. Juicy - Noerre

1951. Lucifer
1952. Manons Restaurant
1953. Restaurant Eldorado
1954. Halmtorvet 9
1955. Kongedybet Brunch and Dinner
1956. cafe Emil
1957. Cafe B
1958. Sushiya
1959. Sicilia Pizzeria
1960. Vintage Coffee
1961. Hansens Is Kaffe
1962. Kebaben
1963. Thaiaway
1964. Pizza House
1965. Indian Tandoor Restaurant
1966. DFDS Restaurant Little Italy
1967. M.G. Petersens Familiehave
1968. Jin Cheng
1969. Nubilo Pizza
1970. Restaurant Laura
1971. Aroii Thai Takeaway
1972. Letz Sushi Islands Brygge
1973. Cafe Bopa
1974. Restaurant Royal Palace
1975. Barock
1976. Mocca Kaffebar
1977. Restaurant Barcelona
1978. Cafe Phenix
1979. Go Away
1980. Dragon Grill Express
tried next in finding all
1981. Yolkie Egg Joint
1982. Vapiano Tivoli
1983. Chuck & Cherry
1984. Original Coffee Nordhavn
1985. Asian Palace
1986. Giin Khao
1987. Musik-conditoriet
1988. Just Thai
1989. McDonalds
1990. Palermo Pizza
1991. 7/11
1992. Grab ’n' Go Burgers
1993. Indian Flavours
1994. La Pausa
1995. Dorfs Gastrobar
1996. Vivaldi 

2341. Cafe Gologolo
2342. Esperia Pizzabar 2000
2343. Cafe Juicy
2344. Pizza 13
2345. Sushi Innovative
2346. Loop Food
tried next in finding all
no next in finding all
Total restaurant count: 2346


In [15]:
# driver = webdriver.Chrome('chromedriver.exe')
# This should be set to the path of the folder holding the python script.
    
def get_reviews(url):
    print(url)
    #if you want to scrape restaurants info

#     scrapeRestaurantInfo(url)

    nextPage = True
    while nextPage:
        #Requests
        driver.get(url)
        time.sleep(1)
        #Click More button
#         more = driver.find_elements_by_xpath("//span[contains(text(),'More')]")
        more = driver.find_elements_by_xpath("//span[@class='taLnk ulBlueLinks'][contains(text(),'More')]")
        # Push all buttons that unclude the "More" option on each review.
        for x in range(0,len(more)):
            try:
                driver.execute_script("arguments[0].click();", more[x])
                time.sleep(3)
            except:
                pass
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        #Store name
        storeName = soup.find('h1', class_='_3a1XQ88S').text
        #Reviews
        results = soup.find('div', class_='listContainer hide-more-mobile')
        try:
            reviews = results.find_all('div', class_='prw_rup prw_reviews_review_resp')
        except Exception:
            continue
        #Export to csv
        try:
            with open("TripReviews620-720.csv", mode='a', encoding="utf-8") as trip_data:
                data_writer = csv.writer(trip_data, delimiter = ',', quotechar = '"', quoting = csv.QUOTE_MINIMAL)
                for review in reviews:
                    ratingDate = review.find('span', class_='ratingDate').get('title')
                    reviewHeader = review.find('span', class_='noQuotes').text
                    text_review = review.find('p', class_='partial_entry')
                    if len(text_review.contents) > 2:
                        reviewText = str(text_review.contents[0][:-3]) + ' ' + str(text_review.contents[1].text)
                    else:
                        reviewText = text_review.text
                    reviewerUsername = review.find('div', class_='info_text pointer_cursor')
                    reviewerUsername = reviewerUsername.select('div > div')[0].get_text(strip=True)
                    rating = review.find('div', class_='ui_column is-9').findChildren('span')
                    rating = str(rating[0]).split('_')[3].split('0')[0]
                    data_writer.writerow([storeName, reviewerUsername, ratingDate, reviewHeader, reviewText, rating])
        except:
            pass

        #Go to next page if exists
        try:
            unModifiedUrl = str(soup.find('div', class_ = 'prw_rup prw_common_responsive_pagination').find('a', class_='nav next ui_button primary', href=True).get('href'))
            # unModifiedUrl = str(soup.find('a', class_ = 'nav next ui_button primary',href=True)['href'])
            url = 'https://www.tripadvisor.com' + unModifiedUrl
        except:
            nextPage = False


In [16]:
with open("AllRestaurants.txt", "rb") as f:   # Unpickling
    urls = pickle.load(f)
print(urls[0])

https://www.tripadvisor.com/Restaurant_Review-g189541-d19608198-Reviews-Maple_Casual_Dining-Copenhagen_Zealand.html


In [36]:
with open("urls_left.txt", "rb") as f:   # Unpickling
    urls_left = pickle.load(f)

# driver_path = f'{os.getcwd()}/chromedriver'
# driver = webdriver.Chrome(driver_path)

finished = []
rest_100 = urls_left[55:100]
for url in rest_100:
    get_reviews(url)
    finished.append(url)

https://www.tripadvisor.com/Restaurant_Review-g189541-d15334999-Reviews-Godtfolk-Copenhagen_Zealand.html
https://www.tripadvisor.com/Restaurant_Review-g189541-d12965339-Reviews-Midori_Izakaya_and_Greenery-Copenhagen_Zealand.html
https://www.tripadvisor.com/Restaurant_Review-g189541-d12909929-Reviews-Terra-Copenhagen_Zealand.html
https://www.tripadvisor.com/Restaurant_Review-g189541-d7215227-Reviews-Sushi_Lovers-Copenhagen_Zealand.html
https://www.tripadvisor.com/Restaurant_Review-g189541-d4131789-Reviews-Cafe_Fremtiden-Copenhagen_Zealand.html
https://www.tripadvisor.com/Restaurant_Review-g189541-d2436505-Reviews-Restaurant_Gilleleje-Copenhagen_Zealand.html
https://www.tripadvisor.com/Restaurant_Review-g189541-d1098090-Reviews-Restaurant_Groften-Copenhagen_Zealand.html
https://www.tripadvisor.com/Restaurant_Review-g189541-d694995-Reviews-Soren_K-Copenhagen_Zealand.html
https://www.tripadvisor.com/Restaurant_Review-g189541-d6499734-Reviews-Hot_Buns-Copenhagen_Zealand.html
https://www.tri

In [37]:
remove = [urls_left.index(finished[0]), urls_left.index(finished[-1])]
print(remove)

[55, 99]


In [38]:
del urls_left[55:100]

In [39]:
# write out the urls that are left to scrape
with open('urls_left.txt', 'wb') as f:
    pickle.dump(urls_left, f)